In [24]:
from langchain_core.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate

In [25]:
examples=[
  {"input":"2+2","output":"4"},
  {"input":"2+3","output":"5"}
]

In [26]:
example_prompt=ChatPromptTemplate.from_messages(
    [
      ("human","{input}"),
      ("ai","{output}"),
    ]
)

few_shot_prompt=FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [27]:
 final_prompt=ChatPromptTemplate.from_messages(
   [
     ("system","你是一个非常厉害的数学天才"),
     few_shot_prompt,
     ("human","{input}"),
   ]
 )

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

output_parser=StrOutputParser()

chain=final_prompt | chat | output_parser

chain.invoke({"input":"3的平方是多少？"})

'9'

动态几次提示

In [29]:
# 多样化任务示例
creative_examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "牛对月亮说了什么？", "output": "什么都没有"},
    {
        "input": "给我写一首关于月亮的五言诗",
        "output": "月儿挂枝头，清辉洒人间。银盘如明镜，照亮夜归人。思绪随风舞，共赏中秋圆。"
    }
]

In [30]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)


to_vectorize=[" ".join(example.values()) for example in creative_examples]
vectorstore=Chroma.from_texts(to_vectorize,embeddings,metadatas=creative_examples)

In [31]:
example_selector=SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"input":"对牛弹琴"})

[{}, {'output': '什么都没有', 'input': '牛对月亮说了什么？'}]

In [33]:
few_shot_prompt=FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    )
)

In [34]:
few_shot_prompt.format(input="What's 3+3?")

'Human: 2+3\nAI: 5\nHuman: 2+3\nAI: 5'

In [35]:
 final_prompt=ChatPromptTemplate.from_messages(
   [
     ("system","你是一个非常厉害的数学天才"),
     few_shot_prompt,
     ("human","{input}"),
   ]
 )


In [36]:
print(final_prompt.format(input="What's 3+3?"))

System: 你是一个非常厉害的数学天才
Human: 2+3
AI: 5
Human: 2+3
AI: 5
Human: What's 3+3?


In [37]:
chain.invoke({"input":"What's 3+5?"})

'8'